In [4]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Bastian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [9]:
from datasets import load_dataset

dataset = load_dataset('GEOcite/ReferenceParserDataset')

Extracting data files: 100%|█████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  9.17it/s]


Dataset ref_seg_ger downloaded and prepared to C:/Users/Bastian/.cache/huggingface/datasets/MrPotato___ref_seg_ger/default/1.0.0/71d78ec91c18c7e5b81314b14f8889c26c4a40b9f053291690f8a5f39c7447c3. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 90.91it/s]


In [10]:
id2label = {k:v for k,v in enumerate(dataset['train'].features['labels'].feature.names)}
label2id = {v:k for k,v in enumerate(dataset['train'].features['labels'].feature.names)}
id2label_ref = {k:v for k,v in enumerate(dataset['train'].features['labels_ref'].feature.names)}
label_ref2id = {v:k for k,v in enumerate(dataset['train'].features['labels_ref'].feature.names)}

In [11]:
dataset["train"][2]

{'id': '10967.xml.csv',
 'tokens': ['[',
  'Bosch',
  '2005',
  ']',
  'In',
  'letzter',
  'Sekunde',
  'vor',
  'dem',
  'Unfall',
  ',',
  'www',
  '.',
  'bosch',
  '.',
  'de',
  '/',
  'start',
  '/',
  'content',
  '/',
  'language',
  '1',
  '/',
  'html',
  '/',
  '715',
  '_',
  '3979',
  '.',
  'htm',
  '(',
  '23',
  '.',
  '12',
  '.',
  '05',
  ')',
  'Aschenbrenner',
  ',',
  'Norbert',
  ',',
  '2005',
  ':',
  'Clevere',
  'Beifahrer',
  ',',
  'in',
  ':',
  'Pictures',
  'of',
  'the',
  'Future',
  ',',
  'Herbst',
  '2005',
  ':',
  '40',
  '-',
  '42',
  'Blutner',
  ',',
  'Doris',
  '/',
  'Stephan',
  'Cramer',
  '/',
  'Tobias',
  'Haertel',
  ',',
  '2006',
  ':',
  'Der',
  'Mensch',
  'als',
  'Problemlöser',
  'in',
  'logistischen',
  'Ketten',
  '(',
  'Universität',
  'Dortmund',
  ',',
  'Soziologische',
  'Arbeitspapiere',
  'Nr',
  '.',
  '13',
  ')',
  'Grell',
  ',',
  'Detlef',
  ',',
  '2003',
  ':',
  'Rad',
  'am',
  'Draht',
  ',',
  'in',
  '

# CRF

In [14]:
def word2features(dataset, i):
    word = dataset["tokens"][i]
    l = dataset["labels"][i]
    #pos = dataset["pos_tags"][i]


    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': pos,
        #'postag[:2]': pos[:2],
    }
    if i > 0:
        word1 = dataset["tokens"][i-1]
        l1 = dataset["labels"][i-1]
        #pos1 = dataset["pos_tags"][i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': pos1,
            #'-1:postag[:2]': pos1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(dataset["tokens"])-1:
        word1 = dataset["tokens"][i+1]
        l1 = dataset["labels"][i+1]
        #pos1 = dataset["pos_tags"][i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': pos1,
            #'+1:postag[:2]': pos1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent["tokens"]))]

def sent2labels(sent):
    return [id2label[label] for label in sent["labels"]]

def sent2tokens(sent):
    return [token for token in sent["tokens"]]

def sent2labels(sent):
    return [id2label[label] for label in sent["labels"]]

#y_train = [sent2labels(s) for s in dataset['train']]

In [15]:
d_train = dataset["train"]
d_test = dataset["test"]

X_train = [sent2features(s) for s in d_train]
y_train = [sent2labels(s) for s in d_train]

X_test = [sent2features(s) for s in d_test]
y_test = [sent2labels(s) for s in d_test]

In [16]:
X_train[0]

[{'bias': 1.0,
  'word.lower()': 'abbink',
  'word[-3:]': 'ink',
  'word[-2:]': 'nk',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': ',',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': ',',
  'word[-3:]': ',',
  'word[-2:]': ',',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': 'abbink',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '+1:word.lower()': 'j',
  '+1:word.istitle()': True,
  '+1:word.isupper()': True},
 {'bias': 1.0,
  'word.lower()': 'j',
  'word[-3:]': 'J',
  'word[-2:]': 'J',
  'word.isupper()': True,
  'word.istitle()': True,
  'word.isdigit()': False,
  '-1:word.lower()': ',',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '+1:word.lower()': '.',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': '.',
  'word[-3:]': '.',
  'word[-2

In [17]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

C:\Users\Bastian\.conda\envs\crf\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

# Evaluation

In [18]:
crf_labels = list(crf.classes_)

In [19]:
y_pred = crf.predict(X_test)


In [20]:
import seqeval
from seqeval.metrics import classification_report

In [23]:
print(classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

      author      0.735     0.752     0.743      4080
      editor      0.500     0.407     0.449       420
       fpage      0.973     0.961     0.967      1188
       issue      0.741     0.674     0.706       267
       lpage      0.972     0.988     0.980      1109
       other      0.771     0.633     0.695       787
   publisher      0.809     0.602     0.690       344
      source      0.791     0.745     0.767      1445
       title      0.787     0.763     0.775      1952
         url      0.729     0.778     0.753        45
      volume      0.906     0.903     0.904      1038
        year      0.960     0.976     0.968      1935

   micro avg      0.826     0.807     0.816     14610
   macro avg      0.806     0.765     0.783     14610
weighted avg      0.824     0.807     0.814     14610



In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
base_filename = "fig_refseg_crf_"

In [ ]:
report = classification_report(y_test, y_pred, output_dict=True)
df_cr = pd.DataFrame(report)
fig, ax = plt.subplots(figsize=(12,10), dpi=300)
sns.heatmap(df_cr.iloc[:-1, :].T, annot=True, fmt=".3f", cmap= 'RdBu_r', vmin=0, vmax=1, ax=ax )
ax.hlines([3], *ax.get_xlim(), colors="white", linewidths = 3.5)
fig.show()
fig.savefig(base_filename + "all_cls_report.png")